## Vectorial Measurement of Coefficients

Here we do the same thing for data obtained vectorially.

To this end, we need to:
- determine coupling coefficient, FWHM, unloaded quality factor, external quality factor, power loss
- can get also length of conductor and compare this with actual length of conductor
- compare between calibrated and uncalibrated results 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os

from utils import dB_to_U

# modify this based on where you open the notebook
PATH_TO_DATA = os.path.join(os.getcwd(), "data")

# data has the following columns (for each column): resonant freq | rho at resonance | rho(omega0) unc | phase offset | kappa| rho at half fwhm | FWHM
data = np.genfromtxt(os.path.join(PATH_TO_DATA, "vector.csv"), delimiter=",", missing_values=np.nan, filling_values=np.nan, skip_header=1, usecols=(0,1,2,3,5,6,7))

uncal_data = data[0]
cal_data = data[1]

In [3]:
uncal_data

array([ 2.99236253e+00,  4.43900000e+02,  5.00000000e-01,  2.80000000e+01,
       -1.00451000e+00,  6.24400000e+02,  3.00150070e+00])

In [4]:
# define bunch of functions to evaluate relevant quantities

def coup_coeff(rho_res):
    '''coupling coefficient, evaluated from reflection coeff. at resonant frequency'''
    # need to have both positive and negative since we 
    # cannot tell if rho is >0 or <0 using scalar measurement
    kappa_pos = (1 + np.abs(rho_res)) / (1 - np.abs(rho_res))
    kappa_neg = 1 / kappa_pos

    return kappa_pos, kappa_neg

def rho_res_kappa(kappa):
    '''reflection coeff. at resonance evaluated from coupling coefficient. Used for verification.'''
    return np.abs((kappa - 1) / (kappa + 1))

def refl_hfwhm(kappa):
    '''reflection coefficient at half of FWHM using coupling coefficient'''
    return np.sqrt(kappa**2. + 1) / (kappa + 1)

def kappa_refl_hfwhm(rho_hfwhm):
    '''Coupling coefficient from rho at half-FWHM. Use to check'''
    kappa_pos = (-rho_hfwhm**2. + np.sqrt(2 * rho_hfwhm**2. + 1) ) / (rho_hfwhm**2. - 1)
    kappa_neg = (-rho_hfwhm**2. - np.sqrt(2 * rho_hfwhm**2. + 1) ) / (rho_hfwhm**2. - 1)
    return kappa_pos, kappa_neg

def Q_factor(res_freq, fwhm_freq):
    '''Loaded quality factor from resonant frequency and half of FWHM'''
    return res_freq / fwhm_freq

def Q0_factor(Q, kappa):
    '''Unloaded quality factor from loaded one and coupling coefficient'''
    return np.abs(1 + np.abs(kappa)) * Q

def Qext_factor(Q0, kappa):
    '''External quality factor'''
    return Q0 / np.abs(kappa)

def standing_wave_ratio(rho):
    '''Standing wave ratio in terms of reflection coefficient'''
    return np.abs(( 1 + np.abs(rho)) / (1 - np.abs(rho)))

def power_loss_ratio(swr):
    '''Ratio of power dissipated to input power'''
    return 4 * swr / (1 + swr)**2.

def length(phase, Q):
    # convert phase
    phase_rad = phase * np.pi / 180.
    return phase_rad * Q / (2* np.pi)

In [5]:
# error propagation of kappa, rho(half fwhm), Q, Q0

def kappa_err(rho, rho_err):
    return 2 * rho_err / (1 + rho)**2.

def refl_hfwhm_err(kappa, kappa_err):
    return (kappa - 1) * kappa_err / ( (1 + kappa) * np.sqrt(kappa**2. + 1) )

def Q_err(res_freq, res_freq_err, half_fwhm, half_fwhm_err):
    return np.sqrt((res_freq_err / half_fwhm)**2. + (res_freq * half_fwhm_err / half_fwhm**2. )**2.)

def Q0_err(Q, Q_err, kappa, kappa_err):
    return np.sqrt( (Q * kappa_err)**2. + (np.abs(1 + kappa) * Q_err)**2.)

def Qext_err(Q0, Q0_err, kappa, kappa_err):
    return np.sqrt(Q0_err**2. + (Q0 / kappa * kappa_err)**2. ) / np.abs(kappa)

def swr_err(rho, rho_err):
    return 2 * rho_err / (1-np.abs(rho))**2.

def Ploss_err(swr, swr_err):
    return np.sqrt((4 * (1 - swr) * swr_err / (1 + swr)**3.)**2.)

## Evaluation of relevant quantities
- Q-factors
- FWHM
- power loss
- length of conductor


Note: for the uncalibrated results, we got the wrong value of $\rho(\Delta \omega)$ due to input error (we input wrong value of kappa) and thus the FWHM for uncalibrated is wrong.

We should document this, but we can also evaluate what the actual value was supposed to be. We will work with the wrong values for now.

In [6]:
'''What we got for uncalibrated data'''

# minimal stepsize in VNA
freq_uncal_err = 0.0000050  # GHz

# all parameters in variables
res_freq_uncal, rho_res_uncal, rho_res_uncal_err, phase_uncal, \
    kappa_uncal, rho_hfwhm_uncal, fwhm_uncal = uncal_data

# convert fwhm to GHz
fwhm_uncal *= 1e-3

# convert reflection coefficient to U
# rho_res_uncal *= 1e-3
# rho_res_uncal_err *= 1e-3
# rho_hfwhm_uncal *= 1e-3

# evaluate necessary variables
# check if we get same kappa_uncal, rho(half FWHM) as in lab
kappa_uncal_eval = coup_coeff(rho_res_uncal)
print(kappa_uncal_eval, kappa_uncal)

rho_hfwhm_uncal_eval = refl_hfwhm(kappa_uncal_eval[1])
# print(rho_hfwhm_uncal, rho_hfwhm_uncal_eval)
print("Kappa, rho at half-FWHM", kappa_uncal, rho_hfwhm_uncal)
print("FWHM", fwhm_uncal)

# evaluate quantities
Q_uncal = Q_factor(res_freq_uncal, fwhm_uncal)
Q0_uncal = Q0_factor(Q_uncal, kappa_uncal)
Qext_uncal = Qext_factor(Q0_uncal, kappa_uncal)

# errors
kappa_uncal_unc = kappa_err(rho_res_uncal, rho_res_uncal_err)
Q_uncal_unc = Q_err(res_freq_uncal, freq_uncal_err, fwhm_uncal, np.sqrt(2) * freq_uncal_err)
Q0_uncal_unc = Q0_err(Q_uncal, Q_uncal_unc, kappa_uncal, kappa_uncal_unc)
Qext_uncal_unc = Qext_err(Q0_uncal, Q0_uncal_unc, kappa_uncal, kappa_uncal_unc)

print("kappa: {0} +- {1}".format(kappa_uncal, kappa_uncal_unc))
print("Q: {0} +- {1}, Q0: {2} +- {3}, Qext: {4} +- {5}: ".format(Q_uncal, Q_uncal_unc, Q0_uncal, Q0_uncal_unc, Qext_uncal, Qext_uncal_unc))

swr_uncal = standing_wave_ratio(rho_res_uncal)
swr_uncal_err = swr_err(rho_res_uncal, rho_res_uncal_err)
print("Standing wave ratio: {0} +- {1}".format(swr_uncal, swr_uncal_err))

Ploss_uncal = power_loss_ratio(swr_uncal)
Ploss_uncal_err = Ploss_err(swr_uncal, swr_uncal_err)
print("Power loss ratio: {0} +- {1}".format(Ploss_uncal, Ploss_uncal_err))

P0, P0_err = dB_to_U(1.5, 0.1)  # reference point of 1mW
print("Power loss: {0} +- {1}".format(Ploss_uncal*P0 * 0.001, np.sqrt(Ploss_uncal_err**2. + (P0_err * 0.001)**2.)))
# l_uncal = length(phase_uncal, Q_uncal)
# print("Length of conductor: ", l_uncal / 1.5)

(-1.0045156920298035, -0.9955046077770286) -1.00451
Kappa, rho at half-FWHM -1.00451 624.4
FWHM 0.0030015007
kappa: -1.00451 +- 5.052137809588059e-06
Q: 996.955464649067 +- 2.3486722722098987, Q0: 1998.4071984437012 +- 0.01172903339446619, Qext: 1989.4348472824572 +- 0.015377034114644087: 
Standing wave ratio: 1.0045156920298035 +- 5.097868627007866e-06
Power loss ratio: 0.9999949250740467 +- 1.1432588314990051e-08
Power loss: 0.0015673005836883051 +- 2.347618662822882e-05


In [9]:
'''What we got for uncalibrated data'''

# minimal stepsize in VNA
freq_cal_err = 0.0000050  # GHz

# all parameters in variables
res_freq_cal, rho_res_cal, rho_res_cal_err, phase_cal, \
    kappa_cal, rho_hfwhm_cal, fwhm_cal = cal_data

# convert fwhm to GHz
fwhm_cal *= 1e-3
print(fwhm_cal)
# convert reflection coefficient to U
# rho_res_cal *= 1e-3
# rho_res_cal_err *= 1e-3
# rho_hfwhm_cal *= 1e-3

# evaluate necessary variables
# check if we get same kappa_cal, rho(half FWHM) as in lab
kappa_cal_eval = coup_coeff(rho_res_cal)
print(kappa_cal_eval, kappa_cal)

rho_hfwhm_cal_eval = refl_hfwhm(kappa_cal_eval[1])
# print(rho_hfwhm_cal, rho_hfwhm_cal_eval)
print("Kappa, rho at half-FWHM", kappa_cal, rho_hfwhm_cal)

# evaluate quantities
Q_cal = Q_factor(res_freq_cal, fwhm_cal)
Q0_cal = Q0_factor(Q_cal, kappa_cal)
Qext_cal = Qext_factor(Q0_cal, kappa_cal)

# errors
kappa_cal_unc = kappa_err(rho_res_cal, rho_res_cal_err)
Q_cal_unc = Q_err(res_freq_cal, freq_cal_err, fwhm_cal, np.sqrt(2) * freq_cal_err)
Q0_cal_unc = Q0_err(Q_cal, Q_cal_unc, kappa_cal, kappa_cal_unc)
Qext_cal_unc = Qext_err(Q0_cal, Q0_cal_unc, kappa_cal, kappa_cal_unc)

print("kappa: {0} +- {1}".format(kappa_cal, kappa_cal_unc))
print("Q: {0} +- {1}, Q0: {2} +- {3}, Qext: {4} +- {5}: ".format(Q_cal, Q_cal_unc, Q0_cal, Q0_cal_unc, Qext_cal, Qext_cal_unc))

swr_cal = standing_wave_ratio(np.abs(rho_res_cal))
swr_cal_err = swr_err(rho_res_cal, rho_res_cal_err)
print("Standing wave ratio: {0} +- {1}".format(swr_cal, swr_cal_err))

Ploss_cal = power_loss_ratio(swr_cal)
Ploss_cal_err = Ploss_err(swr_cal, swr_cal_err)
print("Power loss ratio: {0} +- {1}".format(Ploss_cal, Ploss_cal_err))

P0, P0_err = dB_to_U(1.5, 0.1)  # reference point of 1mW
print("Power loss / mW: {0} +- {1}".format(Ploss_uncal*P0, np.sqrt(Ploss_uncal_err**2. + (P0_err)**2.)))

# l_cal = length(phase_cal, Q_cal)
# print("Length of conductor: ", l_cal / 1.5)

0.0022187499
(-1.0034470872113064, -0.9965647543799382) -1.498
Kappa, rho at half-FWHM -1.498 410.97
kappa: -1.498 +- 2.950228117538504e-06
Q: 1348.3099199238272 +- 4.2970107565032825, Q0: 3368.0781799697206 +- 2.1399150538667535, Qext: 2248.3832977100938 +- 1.428521585183922: 
Standing wave ratio: 1.0034470872113064 +- 2.9706025605881123e-06
Power loss ratio: 0.999997039610953 +- 5.093580604307629e-09
Power loss / mW: 1.5673005836883052 +- 0.023476183844472778


Below is what we should have gotten for uncalibrated data. We get the FWHM from interpolating the plot $|\rho| vs \Delta\omega$ such that $|\rho| = |\rho_{FWHM}|$. 

In [12]:
# '''What we should have gotten for uncalibrated data'''

# from scipy.interpolate import interp1d

# # minimal stepsize in VNA
# freq_err = 0.0000050  # GHz

# # all parameters in variables
# res_freq_uncal_true, rho_res_uncal_true, rho_res_uncal_true_err, phase_uncal_true, _, _, _ = uncal_data

# # convert reflection coefficient to U
# # rho_res_uncal *= 1e-3
# # rho_res_uncal_err *= 1e-3
# # rho_hfwhm_uncal *= 1e-3

# # evaluate necessary variables
# # check if we get same kappa_uncal, rho(half FWHM) as in lab
# kappa_uncal_true_eval = coup_coeff(rho_res_uncal_true)
# print(kappa_uncal_true_eval)

# rho_hfwhm_uncal_true_eval = refl_hfwhm(kappa_uncal_true_eval[1])

# # now start interpolation procedure

# # 
# # print(rho_hfwhm_uncal_true, rho_hfwhm_uncal_eval)

# # # evaluate quantities
# # Q_uncal = Q_factor(res_freq_uncal, fwhm_uncal)
# # Q0_uncal = Q0_factor(Q_uncal, kappa_uncal)
# # Qext_uncal = Qext_factor(Q0_uncal, kappa_uncal)

# # print("Q, Q0, Qext: ", Q_uncal, Q0_uncal, Qext_uncal)

# # swr_uncal = standing_wave_ratio(rho_res_uncal)
# # print("Standing wave ratio: ", swr_uncal)

# # Ploss_uncal = 4 * swr_uncal / ( 1 + swr_uncal)**2.
# # print("Power loss ratio: ", Ploss_uncal)

(-1.0045156920298035, -0.9955046077770286)


In [7]:
# # FWHM for uncalibrated (from image)
# fwhm_uncal = 2 * (2.993062875188 - 2.991562124812)

# # FWHM for calibrated (from image)
# fwhm_cal = 2.9914375 - 2.99365625

0.003001500752000119

-0.002218749999999936

-1.0022674882097262